In [30]:
from seqnmf import seqnmf, plot, example_data
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter1d
from numpy.linalg import norm
from scipy.signal import convolve
from scipy.signal import decimate
import pickle

import time

import mne, os, ast, logging
import pandas as pd
import numpy as np
from os.path import join
from matplotlib import pyplot as plt

from expt_params import *
from functions import *



In [2]:
emu = 'EMU128'

if os.path.exists(params['outdir_edfs']):
    power_cube = np.load(params['outdir_edfs'] + '%s-power_cube.npy' % emu)
else:
    power_cube = np.load(params['outdir_edfs_WIN']  + '%s-power_cube.npy' % emu)


In [3]:
#We are struggling with the immense amount of data. Let's try smoothing the power values, then downsampling, then standardizing (below)
sigma = 20
#Later, individual sigmas for each band and do for all bands. 
for ch in range(power_cube.shape[0]):
    if ch%10==0:print("Working on Channel #%s" % ch)
    for band in range(2, 3):
        power_cube[ch, band, :] = gaussian_filter1d(power_cube[ch, band, :], sigma=sigma, axis=-1)

Working on Channel #0
Working on Channel #10
Working on Channel #20
Working on Channel #30
Working on Channel #40
Working on Channel #50
Working on Channel #60
Working on Channel #70
Working on Channel #80
Working on Channel #90
Working on Channel #100
Working on Channel #110


In [4]:
# Now we downsample a bit...

ds_factor = 10
decimated = decimate(power_cube[:, 2, :], ds_factor, axis=-1, ftype='fir')


In [9]:
print(decimated[:, :].shape)
print(power_cube.shape)
del power_cube

(119, 360000)
(119, 6, 3600000)


In [12]:

### Let's do a sliding z-score on this data for each channel so that we can compare fairly across them. No high power channels will dominate motifs!

def sliding_zscore(arr, window_size):
    mean = uniform_filter1d(arr, size=window_size, mode='nearest')
    sq_mean = uniform_filter1d(arr**2, size=window_size, mode='nearest')
    std = np.sqrt(sq_mean - mean**2)
    z = (arr - mean) / std
    return z


def legacy_sliding_zscore(power_cube, bin_width=1000, target_band=None):
    """
    Applies a sliding z-score normalization to a 3D power cube (channels x bands x time).
    Only normalizes a specific frequency band if target_band is specified (int).
    """
    normalized_power_cube = np.zeros_like(power_cube)
    half_bin = bin_width // 2
    n_channels, n_bands, n_times = power_cube.shape

    for ch in range(n_channels):
        print(f"Working on Channel {ch}")
        for band in range(n_bands):
            if (target_band is not None) and (band != target_band):
                continue
            for i in range(n_times):
                start = max(0, i - half_bin)
                end = min(n_times, i + half_bin)
                window = power_cube[ch, band, start:end]

                data_mean = np.mean(window)
                data_std = np.std(window)
                if data_std == 0:
                    print(f"  ⚠️ Std=0 at ch={ch}, band={band}, time={i}")
                    data_std = 1e-6  # Prevent divide-by-zero

                normalized_power_cube[ch, band, i] = (power_cube[ch, band, i] - data_mean) / data_std

    return normalized_power_cube


#Change this to decimated...
normalized_power_cube = np.zeros_like(decimated)
bin_width = 100

for ch in range(decimated.shape[0]):
    if ch %10 ==0: print("Current Channel is %s" % ch);
    normalized_power_cube[ch, :] = sliding_zscore(decimated[ch, :], bin_width)
"""
#For power cube:
normalized_power_cube = np.zeros_like(power_cube)
bin_width = 1000  # Must be odd to center the window (if not, close enough is fine) 

for ch in range(power_cube.shape[0]): #power_cube.shape[0]cha
    print(f"Channel {ch}")
    for band in range(power_cube.shape[1]):
        normalized_power_cube[ch, band, :] = sliding_zscore(power_cube[ch, band, :], bin_width)
"""
#Legacy, works but is extremely slow computationally.
#print(np.allclose(test_normPC[0, 0, 10000:20000], normalized_power_cube[0, 0, 10000:20000], atol=1e-5))  # Should be True


Current Channel is 0
Current Channel is 10
Current Channel is 20
Current Channel is 30
Current Channel is 40
Current Channel is 50
Current Channel is 60
Current Channel is 70
Current Channel is 80
Current Channel is 90
Current Channel is 100
Current Channel is 110


'\n#For power cube:\nnormalized_power_cube = np.zeros_like(power_cube)\nbin_width = 1000  # Must be odd to center the window (if not, close enough is fine) \n\nfor ch in range(power_cube.shape[0]): #power_cube.shape[0]cha\n    print(f"Channel {ch}")\n    for band in range(power_cube.shape[1]):\n        normalized_power_cube[ch, band, :] = sliding_zscore(power_cube[ch, band, :], bin_width)\n'

In [15]:
#FOR DECIM

for ch in range(normalized_power_cube.shape[0]):
    if ch%10==0: print(ch)
    min_val = np.min(normalized_power_cube[ch, :])
    normalized_power_cube[ch, :] -= min_val



0
10
20
30
40
50
60
70
80
90
100
110


In [4]:
#FOR POWER_CUBE
#Now we work to make all values positive:
for ch in range(normalized_power_cube.shape[0]):
    print(ch)
    for band in range(normalized_power_cube.shape[1]):
        min_val = np.min(normalized_power_cube[ch, band, :])
        normalized_power_cube[ch, band, :] -= min_val



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118


In [14]:
start_time = time.time()

K = 28
L = 975
Lambda = 0.0005

[W, H, cost, loadings, power] = seqnmf(normalized_power_cube[:, 2, :], K, L=1000, Lambda=Lambda)

end_time = time.time()
runtime_seconds = end_time - start_time
print("Runtime was %s seconds" %runtime_seconds)

MemoryError: Unable to allocate 3.19 GiB for an array with shape (119, 3602000) and data type float64

In [19]:
#FOR DECIM
start_time = time.time()

K = 28
L = 98 #980 ms
Lambda = 0.0005

[W, H, cost, loadings, power] = seqnmf(normalized_power_cube, K, L=L, Lambda=Lambda)

end_time = time.time()
runtime_seconds = end_time - start_time
print("Runtime was %s seconds" %runtime_seconds)

Runtime was 16816.81442117691 seconds


In [29]:
with open("C:\\Users\\Kamron\\Documents\\PhD\\sEEG_processing\\DATA\\motif_test.pkl", "wb") as f:
    pickle.dump([W, H, cost, loadings, power], f)

In [32]:
#Our next step will be to reconstruct the data,
#Then depict how much variance each motif captures and in total all the motifs
#Then we should practice visualizing dynamics of motifs with brain images
##THENN!! We will have all of the tools to 'evaluate' the motifs well, so we can run this with new parameters to find an optimal solution or something.

Fraction of Variance Explained (FVE): 0.17345436877542686


'C:\\Users\\Kamron\\Documents\\PhD\\sEEG_processing\\DATA\\EDFs\\'

(119, 360000)